In [1]:
####################################################################
# Machine Learning 2
####################################################################

# 앙상블 모델 ensemble model - voting
# 여러 모델을 결합하여 성능 높이는 방법이다. 
# voting 은 여러개의 모델이 예측한 값 중에서 다수결로 최종 최종 결과를 
# 정한다. 단, 앙상블 학습을 하게되면 개별 모델에 비해 학습 시간이
# 오래 걸리는 단점이 있다.

from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier  # KNN 불러오기
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

hvc = VotingClassifier(estimators = [('KNN',knn),('SVM',svc),('DT',dtc)], voting='hard')

# 학습
hvc.fit(X_train, y_train)

# 예측
y_hvc_pred = hvc.predict(X_test)

# 평가
hvc_acc = accuracy_score(y_test, y_hvc_pred)
hvc_acc

NameError: name 'knn' is not defined

In [ ]:
# 앙상블 모델 ensemble model - bagging

# Decision Tree 는 한개의 트리를 사용한다. 여러개의 트리를 사용하여 각 모델의 개별
# 예측 값을 보팅을 통해 결정한다. 이처럼 같은 알고리즘 모델을 여러번 돌려 예측하는 방법을
# bagging이라고 한다.  

from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators= 100, max_depth=10, random_state=20)

#학습
rfc.fit(X_train, y_train)

# 예측
y_rfc_pred = rfc.predict(X_test)

# 평가
rfc_acc = accuracy_score(y_test, y_rfc_pred)
rfc_acc

# n_estimators= 50   0.9

In [ ]:
# 앙상블 모델 ensemble model - boosting
# 부스팅은 여러개의 약한 학습기(가벼운 모델)를 순차적으로 학습한다. 
# 잘못 예측한 데이터에 대한 예측 오차를 줄이는 방향으로 모델을 계속 업데이트 한다. 
# 여러 모델을 동시에 학습하지 않고 순서대로 학습하는 점에서 배깅과는 다르다. 

# 캐글, 데이콘 등 경진대회에서 가장 많이 사용되는 알고리즘이다. 
# 모델 학습 속도 빠르고 예측력이 상당히 좋은 편으로 알려져 있다. 
# !pip install xgboost
from xgboost import XGBClassifier

xgbc = XGBClassifier(n_estimators=50, max_depth=3, random_state=20)

#학습
xgbc.fit(X_train, y_train) 

#예측
y_xgbc_pred = xgbc.predict(X_test)

#평가
xgbc_acc = accuracy_score(y_test, y_xgbc_pred)
xgbc_acc



In [ ]:
# 과대 적합 overfitting & 과소 적합 underfitting

# 과소 적합 underfitting - 머신 러닝을 진행할때 학습 시킬 데이터가 
#                          현저히 적을때 예측력이 떨어지는 증상


# 과대 적합 overfitting - 학습 시킨 데이터와 비슷한 것은 예측력이 우수하게 판단하지만
#                         새로운데이터에는 예측력이 떨어지는 증상


# solution : 데이터의 엄청나게 양을 늘린다. 

In [ ]:
# K-fold 교차검증

# 데이터 셋을 5개의 Fold로 분할
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=20)

# 훈련용 데이터와 검증용 데이터의 행 인덱스를 각각의  Fold별로 구분하여 확인해보자

num_fold = 1

for tr_idx, val_idx in kfold.split(X_train):
  print('%s Fold ---------------------------------------' % num_fold)
  print('훈련 :', len(tr_idx), tr_idx[:])
  print('검증 :', len(val_idx), val_idx[:])
  num_fold = num_fold + 1


In [ ]:
# 훈련용 데이터와 검증용 데이터의 행 인덱스를 각 fold별로 구분하여 생성

val_scores = [] # 5번의 예측 정확도를 담을 리스트

num_fold = 1

for tr_idx, val_idx in kfold.split(X_train, y_train):
  # 훈련용 데이터와 검증용 데이터를 행 인덱스 기준으로 추출
  X_tr, X_val = X_train.iloc[tr_idx, :], X_train.iloc[val_idx, :]   #  X 문제지
  y_tr, y_val = y_train.iloc[tr_idx], y_train.iloc[val_idx]         #  y 답안지

  # 학습
  rfc = RandomForestClassifier(max_depth= 5, random_state=20)
  rfc.fit(X_tr,y_tr)

  # 예측
  y_val_pred = rfc.predict(X_val)

  # 검증
  val_acc = accuracy_score(y_val, y_val_pred)

  # 결과 출력
  print('%d Fold Accuracy : %.4f' % (num_fold, val_acc)) # 각각의 폴드마다의 정확도
  val_scores.append(val_acc)
  num_fold += 1

# 평균 정확도 계산

mean_score = np.mean(val_scores)
print()
print('평균 정확도 : ',np.round(mean_score, 2) * 100, '%')
